# AI Teacher-Student Conversation Generator

This notebook creates realistic conversations between AI teachers and students. It's like having different types of teachers talk to different types of students about school subjects.

## What This Does

1. **Reads Educational Materials**: Takes textbooks or lesson materials and understands what they're about
2. **Creates AI Teachers & Students**: Makes different teacher personalities (strict, friendly, etc.) and student types (focused, creative, etc.)
3. **Generates Conversations**: Has these AI characters talk to each other about the lesson topics
4. **Saves the Conversations**: Records everything so it can be turned into audio later

## Cool Features

- **Different Student Types**: Some are good at math, others are creative, some need more help
- **Different Teacher Styles**: Some are encouraging, others ask tough questions, some tell stories
- **Natural Conversations**: The discussions flow like real classroom interactions
- **Audio Ready**: Everything is set up to become spoken conversations

## Who Made This & Why

**Created by**: Allan C. Tan, Jan Murillo  
**Company**: Predictive Systems, Inc.

### What This Project Does

This notebook helps create realistic conversations between AI teachers and students. Each AI character has its own personality and way of teaching or learning.

### Why This Is Useful

- **Helps Different Learners**: Works with students who learn in different ways
- **Tests Teaching Methods**: Tries out different ways teachers can ask questions and give feedback
- **Creates Audio Content**: Makes conversations that can be turned into spoken audio
- **Works Offline**: Can run without internet once set up

### How People Can Use This

- **Make Learning Materials**: Create practice conversations for any subject
- **Train Teachers**: Show examples of different teaching styles
- **Help Students Practice**: Give students different types of questions to work with
- **Build Voice Apps**: Create educational apps that can talk to students

## What You Need to Run This

### Required Software Packages

This section loads all the tools needed to make the AI system work:

- **Basic Tools**: `os`, `time`, `logging` - help the computer manage files and track what's happening
- **Data Organization**: `pydantic`, `dataclasses` - keep information organized and structured
- **AI Brain**: `pydantic_ai` with Google Gemini - this is the smart part that makes the AI teachers and students
- **Settings**: `dotenv` - manages secret passwords and settings
- **Tracking**: `logfire` - keeps track of what the system is doing for debugging

### Setup Requirements

You need these settings configured:
- `LOGFIRE_TOKEN`: For monitoring what the system is doing
- Google AI API key: This lets you use Google's AI to power the conversations

In [ ]:
import os
import logfire
import time
from logging import getLogger
from pydantic import BaseModel, Field
from dotenv import load_dotenv
from typing import Annotated, List, Optional
from typing_extensions import TypedDict
from dataclasses import dataclass, field
from pydantic_ai import Agent,RunContext, DocumentUrl
from pydantic_ai.models.google import GoogleProvider
from operator import add

from dataclasses import dataclass
from pydantic_graph import BaseNode, GraphRunContext

load_dotenv()

logfire.configure(token=os.getenv('LOGFIRE_TOKEN'))  
logfire.instrument_pydantic_ai()  

logger = getLogger(__name__)

Logfire project URL: https://logfire-us.pydantic.dev/itsybitsci/starter-project
Currently retrying 1 failed export(s)


In [2]:
agent = Agent(
    'google-gla:gemini-2.5-flash',
    system_prompt='Be concise, reply with one sentence.',
)

## Testing the AI System

### Quick Test to Make Sure Everything Works

This section checks that the AI system is working properly. It tests:

1. **Connection**: Makes sure we can connect to Google's AI
2. **Basic Talking**: Tests if the AI can answer simple questions
3. **Good Answers**: Checks that the AI gives helpful, short responses

**What Should Happen**: The AI should give a quick, accurate answer about where "Hello, World!" comes from in programming.

In [3]:
result = await agent.run('Where does "hello world" come from?')
print(result.output)
"""
The first known use of "hello, world" was in a 1974 textbook about the C programming language.
"""

16:06:51.041 agent run
16:06:51.043   chat gemini-2.5-flash
It originated as a simple test program in Brian Kernighan's 1974 tutorial for the B programming language and was popularized by *The C Programming Language* book.


'\nThe first known use of "hello, world" was in a 1974 textbook about the C programming language.\n'

## How the Data is Organized

### The Building Blocks of Our System

Here's a diagram showing how all the pieces fit together:

![Data Models](./images/assessment.png)

### Main Parts

- **Topics**: Individual lesson subjects with their content
- **Questions**: The actual questions we ask, plus follow-up questions and answer guidelines
- **Assessments**: Groups of questions that go together for testing a topic
- **Student Personalities**: Descriptions of different types of students (shy, confident, etc.)
- **Teacher Styles**: Different ways teachers can ask questions and give feedback

### The Code That Defines Our Data

The code below creates the templates for organizing all our information. It makes sure everything is structured consistently and works well together.

#### What this code creates:

1. **Topics**: Templates for lesson content and descriptions
2. **Questions**: Templates for main questions and follow-up questions with answer guidelines
3. **Assessments**: Templates for complete sets of questions
4. **System State**: Keeps track of what the system is doing
5. **Dependencies**: Connects different parts of the system together

In [4]:

class AssessmentTopic(BaseModel):
    title: str = Field(description="Concise title of the topic section")
    description: str = Field(description="Description of the topic section")
    content: str = Field(description="Content of the topic section")

criterion = Annotated[
    str,
    Field(
        description="Criteria for evaluating the answer", min_length=5, max_length=150
    ),
]

class FollowUpQuestion(BaseModel):
    text: str = Field(
        description="Text of the follow-up question", min_length=3, max_length=300
    )
    answer: str = Field(
        description="An example answer to the follow-up question",
        min_length=1,
        max_length=300,
    )
    criteria: Annotated[
        list[criterion],
        Field(
            description="List of criteria for evaluating the answer",
            min_length=1,
            max_length=5,
        ),
    ]
    isSelected: bool = False


class Question(FollowUpQuestion):
    followUps: List[FollowUpQuestion] = Field(
        description="List of follow-up questions to guide the user in answering the main question"
    )

class Assessment(BaseModel):
    title: str = Field(
        description="Title of the assessment", min_length=3, max_length=100
    )
    description: str = Field(
        description="Description of the assessment", min_length=10, max_length=500
    )
    questions: List[Question] = Field(
        description="List of questions related to the document"
    )

class AssessmentState(TypedDict):
    title: str
    file_url: str
    topics: List[AssessmentTopic]
    assessments: Annotated[List[Assessment], add]
    
class SplitterResult(BaseModel):
    topics: list[AssessmentTopic] = Field(
        description="List of identified topics in the document"
    )
    title: str = Field(description="Title of the document")


@dataclass
class AssessmentDeps:
    title: str
    description: str
    content: str

class AssessmentTopic(BaseModel):
    title: str = Field(description="Concise title of the topic section")
    description: str = Field(description="Description of the topic section")
    content: str = Field(description="Content of the topic section")

    

## Step 1: Reading and Breaking Down Documents

### Setting Up the Document Reader

This section creates an AI that's really good at reading educational materials like textbooks. It breaks long documents into smaller, manageable topics that we can create questions about.

#### What This AI Can Do:

1. **Find Topic Boundaries**: Knows when one topic ends and another begins
2. **Make Good Sections**: Creates chunks that have enough content for several questions
3. **Keep Important Details**: Doesn't lose important scientific terms or explanations
4. **Quality Check**: Makes sure each section can generate at least 5 good questions

#### How It Works:

- **Combines Related Stuff**: Puts related information together instead of splitting it up
- **Avoids Tiny Pieces**: Creates substantial sections rather than lots of tiny ones
- **Keeps Technical Terms**: Doesn't dumb down the scientific language
- **Focuses on Learning**: Organizes content in a way that's good for creating educational questions

In [5]:
splitter_prompt = """You are an expert at analyzing document content and identifying distinct topics. Your task is to split a document into comprehensive topic sections only when there is a clear, substantial change in content, ensuring each topic can support at least 5 extractable questions.

You are also additionally tasked to provide a single unifying title for all the topics, which should be a concise summary of the overall content.

Guidelines:
1. Split only when there is a significant shift in subject or focus.
2. Each topic section must be self-contained and provide enough depth to yield multiple meaningful questions.
3. Do NOT create standalone topics for parts that serve only as context (e.g., datasets, general introductions, background material), practice assessments, pros vs cons, derivations, or summaries. Instead, integrate these elements into related topics if they add value.
4. Group together contextual or background information that supports a larger subject area.
5. Avoid over-splitting; favor fewer, robust sections that capture the overall theme.
6. Think of each topic as a lesson in a book's table of contents. The topics should be neither as broad as an entire module (e.g., "Physics" or "Projectile Motion") nor as narrow as a single detailed discussion (e.g., "Analysis of One Equation"). Each topic should represent a well-defined lesson that is clear, organized, and substantial.
7. Each topic is intended to serve as the basis of an assessment, so ensure that the content is broad and detailed enough to generate at least 5 meaningful questions.
8. Ensure that the topic title itself is not phrased as a question (e.g., avoid titles like "Advantages of X").
9. Avoid too many subtopics, as they can lead to confusion. Instead, focus on creating a few well-defined topics that cover the material comprehensively.

For each topic section:
- Create a concise title (3–5 words) that reflects the main subject.
- Extract the relevant portion of content corresponding to that topic.
- Ensure the extracted content is comprehensive enough to support at least 5 meaningful questions.

You must produce at least 1 topic section, but you can create more if the content allows for it. Each topic section should be clearly defined and distinct from the others.

Your objective is to maintain clarity and cohesion by avoiding the creation of topics for context-only parts, pros vs cons, datasets, derivations, summaries, or practice assessments (which should only serve as inspiration)."""


splitter_agent = Agent(
    model="google-gla:gemini-2.5-flash",
    system_prompt=splitter_prompt,
    output_type=SplitterResult,
)



In [6]:
from pydantic_ai import BinaryContent
import mimetypes

async def split_topics(state: AssessmentState):
    start_time = time.time()
    file_url = state["file_url"]
    logger.info(f"Starting to split file into topics")

    try:
        file_path = state["file_url"]
        with open(file_path, "rb") as f:
            data = f.read()
        mime = mimetypes.guess_type(file_path)[0] or "application/octet-stream"
        content = BinaryContent(data=data, media_type=mime)
        result = await splitter_agent.run(
            [
                "Get the topics from this file and provide a main title for all the topics.",
#                DocumentUrl(url=file_url),
                content
            ]
        )
        title = result.output.title
        topics = result.output.topics
        duration = time.time() - start_time

        logger.info(
            f"Successfully split file titled {title} into {len(topics)} topics (took {duration:.2f}s)"
        )
        if topics:
            logger.debug("Topics identified:")
            for idx, topic in enumerate(topics, 1):
                logger.debug(f"  {idx}. {topic.title}")
                logger.debug(f"     Content length: {len(topic.content)} chars")

        return {"topics": topics, "title": title}
    except Exception as e:
        logger.error(f"Failed to split topics from file {file_url}", exc_info=True)
        raise

result = await split_topics(AssessmentState(file_url="./data/grade_9/science/SCI9-Q4-MOD1-Projectile Motion.pdf"))


16:07:08.738 splitter_agent run
16:07:08.740   chat gemini-2.5-flash


In [7]:
print(f"\n Main Title: {result['title']}\n")
print(f"Found {len(result['topics'])} topics:\n")

for i, topic in enumerate(result['topics'], 1):
    print(f"Topic {i}: {topic.title}")
    print("─" * 40)
    print(f"Description: {topic.description}")
    print(f"Content Preview: {topic.content[:200]}...")
    print()


 Main Title: Projectile Motion: Horizontal and Vertical Components

Found 2 topics:

Topic 1: Introduction to Projectile Motion
────────────────────────────────────────
Description: This topic introduces the fundamental concepts of projectile motion, defining key terms such as projectile and trajectory, and classifying different types of projectile trajectories, including horizontally launched and angle-launched projectiles.
Content Preview: In this module, you will learn the definition of projectile different from projectile motion. You will also learn on how to describe the horizontal and vertical motions of a projectile. And lastly, yo...

Topic 2: Horizontal and Vertical Motion
────────────────────────────────────────
Description: This topic delves into the independent horizontal and vertical components of projectile motion, explaining how each component behaves under the influence of gravity and the absence of horizontal forces for both horizontally and angle-launched projectiles

## Step 2: Creating Questions

### Setting Up the Question Generator

This section creates an AI that's great at turning lesson content into good questions for students. It takes the topics we found and creates structured Q&A sessions.

#### How It Makes Questions:

1. **Reads the Content**: Looks at each topic's information carefully
2. **Creates Main Questions**: Makes primary questions at the right difficulty level
3. **Adds Follow-up Questions**: Creates helpful hints and sub-questions to guide students
4. **Sets Answer Standards**: Decides what makes a good answer
5. **Quality Check**: Makes sure students can actually answer the questions from the content

#### What Each Question Includes:

- **Main Question**: The primary question students need to answer
- **Follow-up Hints**: Helpful prompts that guide students if they're stuck
- **Example Answers**: Shows what a good answer looks like
- **Grading Guidelines**: Clear rules for what counts as correct
- **Simple Text**: Written in a way that works well for voice apps

In [8]:
multi_assessment_prompt = """
    You are given multiple topics, each with a title, content, and description. Your task is to generate an assessment for each topic.
    Each assessment should contain questions based on the content provided. 
    There should be no overlap between the questions of different topics.
    The output should consist of a list of assessments the same amount as the topics provided.
    
    Here is the instruction for generating each individual assessment:
    Given a topic title, content, and description, generate an assessment containing questions using the content provided. 
    1. The file provided serves as context and additional information, but make sure most of the generated data is from the content provided.
    2. The title and description will be the domain of the assessment, while the content will be the basis for the questions.
    3. This will be answered by a student or user.
    The assessment will contain a list of questions that the user must answer.
      Each question must have:
      - A question text
      - An example answer to the question
      - A list of validation rules for the answer, this is called a criteria
      - A list of follow-up questions if the user's answer is not complete or needs more information
  
      Rules for follow-up questions:
      - Each question should have at least one follow-up question
      - A follow-up question is there to help the user answer the main question and nudges the user in the right direction
      - A follow-up question does not introduce new concepts, but rather helps the user understand the main question better
      - Example: If the main question is "What is the acceleration due to gravity?" and the user answers a positive number, a follow-up question could be "Gravity goes down, which means that the answer should be what?"

      Rules for criteria:
      - Should not be too specific
      - Should allow for alternative correct answers based on the PDF
      - The criterion itself should not exceed 150 characters

      Rules for the example answer:
      - Should not exceed 300 characters
  
      Keep the criteria clear and concise. For example, if the answer is a number, you can say "Answer must be number".
  
      Do not include any latex or formatting in any of the response. Make sure that it is plain text only
      and readable by a user. (i.e. if x^2, just say x squared)
      
      Do not include only abbreviations in the answer, make sure that the answer is clear and understandable."""

multi_assessment_agent = Agent(
    model="google-gla:gemini-2.5-flash",
    system_prompt=multi_assessment_prompt,
    output_type=List[Assessment],
    deps_type=List[AssessmentDeps],
)


@multi_assessment_agent.tool
def get_topics(ctx: RunContext[List[AssessmentDeps]]):
    """Get the topics and their respective content."""
    topics = [
        f"[Topic {i}\nTitle: {dep.title}\nDescription: {dep.description}\nContent: {dep.content}"
        for i, dep in enumerate(ctx.deps)
    ]

    return "\n\n".join(topics)

In [9]:
async def generate_multiple_assessments(state: AssessmentState):
    start_time = time.time()
    deps = [AssessmentDeps(**x.model_dump()) for x in state["topics"]]

    logger.info(f"Starting multiple assessment generation")
    logger.debug(f"Number of topics: {len(deps)}")

    try:
        result = await multi_assessment_agent.run(
            [
                "Create assessments for each of the following topics and content",
            ],
            deps=deps,
        )

        duration = time.time() - start_time

        logger.info(f"Generated assessments (took {duration:.2f}s)")

        return {"assessments": result.output}
    except Exception as e:
        logger.error(f"Failed to generate assessments", exc_info=True)
        raise

In [11]:
assessments = await generate_multiple_assessments(AssessmentState(topics=result['topics'], title=result['title']))
assessments

16:09:07.161 multi_assessment_agent run
16:09:07.162   chat gemini-2.5-flash
16:09:08.925   running 1 tool
16:09:08.925     running tool: get_topics
16:09:08.925   chat gemini-2.5-flash


{'assessments': [Assessment(title='Introduction to Projectile Motion', description='This topic introduces the fundamental concepts of projectile motion, defining key terms such as projectile and trajectory, and classifying different types of projectile trajectories, including horizontally launched and angle-launched projectiles.', questions=[Question(text='Define Projectile and Projectile Motion.', answer='A projectile is an object given an initial velocity and acted upon solely by gravity in a curved path. Projectile motion is the form of motion exhibited by such an object.', criteria=['Answer must define both projectile and projectile motion, highlighting the role of initial velocity and gravity.'], isSelected=False, followUps=[FollowUpQuestion(text="What is the primary force acting on a projectile once it's launched?", answer='Gravity is the primary force.', criteria=['Answer must identify gravity as the primary force.'], isSelected=False)]), Question(text='What is a trajectory and 

In [12]:
for i, assessment in enumerate(assessments['assessments'], 1):
    print(f"Title: {assessment.title}")
    print("─" * 40)
    print(f"Description: {assessment.description}") 
    print("─" * 40)
    for question in assessment.questions:
        print(f"Question: {question.text}")
        print(f"Example Answer: {question.answer}")
        print(f"Criteria: {question.criteria}")
        for follow_up in question.followUps:
            print(f"    >> Follow-up Question: {follow_up.text}")
            print(f"    >> Example Answer: {follow_up.answer}")
            print(f"    >> Criteria: {follow_up.criteria}")
        print("─" * 40)
    print()

Title: Introduction to Projectile Motion
────────────────────────────────────────
Description: This topic introduces the fundamental concepts of projectile motion, defining key terms such as projectile and trajectory, and classifying different types of projectile trajectories, including horizontally launched and angle-launched projectiles.
────────────────────────────────────────
Question: Define Projectile and Projectile Motion.
Example Answer: A projectile is an object given an initial velocity and acted upon solely by gravity in a curved path. Projectile motion is the form of motion exhibited by such an object.
Criteria: ['Answer must define both projectile and projectile motion, highlighting the role of initial velocity and gravity.']
    >> Follow-up Question: What is the primary force acting on a projectile once it's launched?
    >> Example Answer: Gravity is the primary force.
    >> Criteria: ['Answer must identify gravity as the primary force.']
──────────────────────────────

## Step 3: Setting Up Conversations

### Building the Conversation System

This section creates the tools needed to manage conversations between AI teachers and students. Think of it like building a chat system that keeps track of who said what and when.

#### Main Parts:

1. **Messages**: Individual things that teachers or students say
2. **Conversation Tracker**: Keeps track of the whole conversation including:
   - Everything that's been said so far
   - What question we're currently on
   - Who's turn it is to talk
   - The conversation history formatted for the AI to understand

#### What This System Can Do:

- **Remember Everything**: Keeps a record of every message in order
- **Format for AI**: Organizes the conversation so the AI can understand what happened
- **Keep Track**: Maintains the flow of conversation across multiple back-and-forth exchanges
- **Know Who's Talking**: Always knows whether it's the teacher or student speaking

In [13]:

@dataclass
class Message:
    who: str
    content: str


@dataclass
class ConversationContext:
    question: Optional[Question] = None             # making these optional for testing
    transcript: list[Message] = field(default_factory=list)

    def getConversationHistory(self) -> str:
        """
        Returns the conversation history as a formatted string for LLM context.
        Each line is formatted as 'Who: message'.
        Only the last 'limit' messages are included.
        """
        history = "\n".join(f"{msg.who}: {msg.content}" for msg in self.transcript)
        return f"<ConversationHistory> {history} </ConversationHistory>"        
        

    def addMessage(self, message: Message):
        self.transcript.append(message)

## Step 4: Creating Different Types of Students

### Making AI Students with Different Personalities

This section creates various AI students, each with their own personality and way of learning. Just like real students, each one is unique and responds differently to questions.

#### What Makes Each Student Different:

1. **Personality**: How outgoing, organized, or creative they are
2. **Learning Style**: Whether they're good with words, numbers, music, etc.
3. **Thinking Style**: How they process information (step-by-step vs. big picture)
4. **Mood**: How they're feeling (excited, nervous, bored, etc.)
5. **Ability Level**: How well they can answer questions (fully, partially, or not at all)

#### Types of Students We Create:

- **Different Strengths**: Some are good with language, others with math, some with art or music
- **Different Approaches**: Some think step-by-step, others see the big picture first
- **Different Energy Levels**: Some are highly motivated, others need more encouragement
- **Different Feelings**: Some are confident, others are anxious or frustrated
- **Different Knowledge**: Some know the material well, others are still learning

In [14]:

from dataclasses import dataclass
from typing import Dict
from enum import Enum

# class LearningModality(Enum):
#     VISUAL = "visual"
#     AUDITORY = "auditory"
#     READING_WRITING = "reading/writing"
#     KINESTHETIC = "kinesthetic"

class DominantIntelligence(str, Enum): # Howard Gardner's Multiple Intelligences
    LINGUISTIC = "linguistic"
    LOGICAL = "logical"
    SPATIAL = "spatial"
    BODILY = "bodily-kinesthetic"
    MUSICAL = "musical"
    INTERPERSONAL = "interpersonal"
    INTRAPERSONAL = "intrapersonal"
    NATURALISTIC = "naturalistic"

class CognitiveStyle(str, Enum):
    SEQUENTIAL = "sequential"
    GLOBAL = "global"
    REFLECTIVE = "reflective"
    ACTIVE = "active"
    INTUITIVE = "intuitive"
    SENSING = "sensing"

class MotivationLevel(str, Enum):
    LOW = "low"
    MEDIUM = "medium"
    HIGH = "high"

class EmotionalState(str, Enum):
    CALM = "calm"
    ANXIOUS = "anxious"
    ENGAGED = "engaged"
    FRUSTRATED = "frustrated"
    BORED = "bored"
    MOTIVATED = "motivated"
    OVERWHELMED = "overwhelmed"

class AnswerAbility(str, Enum):
    CAN_ANSWER = "can fully answer"
    CANNOT_ANSWER = "cannot answer"
    PARTIALLY_ANSWER = "partially answer"

@dataclass
class StudentProfile:
    profile: str
    personality_profile: Dict[str, float]  # e.g., {"introvert": 0.7, "conscientious": 0.9}
    # learning_modality: LearningModality
    dominant_intelligence: DominantIntelligence
    cognitive_style: CognitiveStyle
    motivation_level: MotivationLevel
    emotional_state: EmotionalState
    #answer_ability: int
    answer_ability: AnswerAbility

    def get_description(self) -> str:
        top_traits = sorted(self.personality_profile.items(), key=lambda x: -x[1])
        top_traits_str = ', '.join([f"{trait} ({score:.2f})" for trait, score in top_traits[:3]])
        return (
            f"You are {self.profile}. "
            f"You have {self.dominant_intelligence.value} intelligence, and learn by {self.cognitive_style.value} cognitive style. "
            f"You are currently feeling {self.emotional_state.value} and show a {self.motivation_level.value} level of motivation. "
            f"Your Big 5 personality traits are: {top_traits_str}."
        )

In [15]:
students = [
    StudentProfile(
        profile="a grade 9 high school student who is a focused problem-solver with high logic and structure",
        personality_profile={"openness": 0.65, "conscientiousness": 0.92, "extraversion": 0.3, "agreeableness": 0.6, "neuroticism": 0.2},
        dominant_intelligence=DominantIntelligence.LOGICAL,
        cognitive_style=CognitiveStyle.SEQUENTIAL,
        motivation_level=MotivationLevel.HIGH,
        emotional_state=EmotionalState.ENGAGED,
        answer_ability=AnswerAbility.CAN_ANSWER
    ),
    StudentProfile(
        profile="a grade 9 high school student who is a friendly collaborator who prefers big-picture thinking",
        personality_profile={"openness": 0.75, "conscientiousness": 0.55, "extraversion": 0.8, "agreeableness": 0.9, "neuroticism": 0.3},
        dominant_intelligence=DominantIntelligence.INTERPERSONAL,
        cognitive_style=CognitiveStyle.GLOBAL,
        motivation_level=MotivationLevel.MEDIUM,
        emotional_state=EmotionalState.CALM,
        answer_ability=AnswerAbility.PARTIALLY_ANSWER
    ),
    StudentProfile(
        profile="a grade 9 high school student who is a reflective writer with strong discipline",
        personality_profile={"openness": 0.7, "conscientiousness": 0.95, "extraversion": 0.4, "agreeableness": 0.65, "neuroticism": 0.2},
        dominant_intelligence=DominantIntelligence.LINGUISTIC,
        cognitive_style=CognitiveStyle.REFLECTIVE,
        motivation_level=MotivationLevel.HIGH,
        emotional_state=EmotionalState.MOTIVATED,
        answer_ability=AnswerAbility.CAN_ANSWER
    ),
    StudentProfile(
        profile="a grade 9 high school student who is a creative soul attuned to sound and emotion",
        personality_profile={"openness": 0.9, "conscientiousness": 0.45, "extraversion": 0.6, "agreeableness": 0.7, "neuroticism": 0.6},
        dominant_intelligence=DominantIntelligence.MUSICAL,
        cognitive_style=CognitiveStyle.INTUITIVE,
        motivation_level=MotivationLevel.MEDIUM,
        emotional_state=EmotionalState.ANXIOUS,
        answer_ability=AnswerAbility.CANNOT_ANSWER
    ),
    StudentProfile(
        profile="a grade 9 high school student who is a visual learner who thrives with hands-on exploration",
        personality_profile={"openness": 0.85, "conscientiousness": 0.88, "extraversion": 0.35, "agreeableness": 0.55, "neuroticism": 0.25},
        dominant_intelligence=DominantIntelligence.SPATIAL,
        cognitive_style=CognitiveStyle.ACTIVE,
        motivation_level=MotivationLevel.HIGH,
        emotional_state=EmotionalState.ENGAGED,
        answer_ability=AnswerAbility.CAN_ANSWER
    ),
    StudentProfile(
        profile="a grade 9 high school student who is a deep feeler with self-awareness but low drive",
        personality_profile={"openness": 0.8, "conscientiousness": 0.4, "extraversion": 0.3, "agreeableness": 0.75, "neuroticism": 0.7},
        dominant_intelligence=DominantIntelligence.INTRAPERSONAL,
        cognitive_style=CognitiveStyle.SENSING,
        motivation_level=MotivationLevel.LOW,
        emotional_state=EmotionalState.BORED,
        answer_ability=AnswerAbility.CANNOT_ANSWER
    ),
    StudentProfile(
        profile="a grade 9 high school student who is a curious explorer of nature and patterns",
        personality_profile={"openness": 0.95, "conscientiousness": 0.6, "extraversion": 0.5, "agreeableness": 0.65, "neuroticism": 0.5},
        dominant_intelligence=DominantIntelligence.NATURALISTIC,
        cognitive_style=CognitiveStyle.INTUITIVE,
        motivation_level=MotivationLevel.MEDIUM,
        emotional_state=EmotionalState.OVERWHELMED,
        answer_ability=AnswerAbility.PARTIALLY_ANSWER
    ),
    StudentProfile(
        profile="a grade 9 high school student who is an energetic mover who learns best through action",
        personality_profile={"openness": 0.6, "conscientiousness": 0.5, "extraversion": 0.85, "agreeableness": 0.6, "neuroticism": 0.55},
        dominant_intelligence=DominantIntelligence.BODILY,
        cognitive_style=CognitiveStyle.ACTIVE,
        motivation_level=MotivationLevel.MEDIUM,
        emotional_state=EmotionalState.FRUSTRATED,
        answer_ability=AnswerAbility.CANNOT_ANSWER
    ),
    StudentProfile(
        profile="a grade 9 high school student who tries to sound confident but gives vague and generic answers",
        personality_profile={"openness": 0.4, "conscientiousness": 0.3, "extraversion": 0.7, "agreeableness": 0.8, "neuroticism": 0.5},
        dominant_intelligence=DominantIntelligence.LINGUISTIC, 
        cognitive_style=CognitiveStyle.GLOBAL,  
        motivation_level=MotivationLevel.LOW,   
        emotional_state=EmotionalState.OVERWHELMED,  
        answer_ability=AnswerAbility.PARTIALLY_ANSWER 
    )
]
for student in students:
    print(student.get_description())

You are a grade 9 high school student who is a focused problem-solver with high logic and structure. You have logical intelligence, and learn by sequential cognitive style. You are currently feeling engaged and show a high level of motivation. Your Big 5 personality traits are: conscientiousness (0.92), openness (0.65), agreeableness (0.60).
You are a grade 9 high school student who is a friendly collaborator who prefers big-picture thinking. You have interpersonal intelligence, and learn by global cognitive style. You are currently feeling calm and show a medium level of motivation. Your Big 5 personality traits are: agreeableness (0.90), extraversion (0.80), openness (0.75).
You are a grade 9 high school student who is a reflective writer with strong discipline. You have linguistic intelligence, and learn by reflective cognitive style. You are currently feeling motivated and show a high level of motivation. Your Big 5 personality traits are: conscientiousness (0.95), openness (0.70),

In [16]:

def build_student_prompt(profile: StudentProfile) -> str:
    """
    Turn a StudentProfile into a system prompt the LLM can follow.
    """
    # Use the helper we added earlier
    desc = profile.get_description()
    if profile.answer_ability == AnswerAbility.CAN_ANSWER:
        answer_ability_prompt = "You can fully answer the question."
    elif profile.answer_ability == AnswerAbility.CANNOT_ANSWER:
        answer_ability_prompt = "Do not answer the question. Just say you do not know the answer."
    elif profile.answer_ability == AnswerAbility.PARTIALLY_ANSWER:
        answer_ability_prompt = "You can partially answer the question."

    prompt = f"""
{desc}

**How to respond:**
1. You are being asked by your teacher. Do not use any other names or pronouns. Keep answers short and concise.
2. Respond in a natural voice conversation, matching your personality as a student. 
{answer_ability_prompt}
4. Your tone should reflect your emotional_state (“{profile.emotional_state.value}”).
5. Respond with text for voice utterance only. Do not use any other formatting. Do not include asterisks.
6. If the teacher gives hints, use them to answer the question.
7. Make sure to also answer the follow-up questions.
"""
    return prompt

def build_student_agent(profile: StudentProfile) -> Agent:
    """
    Returns an LLM agent that answers questions as the given student.
    """
    return Agent(
        model="google-gla:gemini-2.5-flash",         # or any model you prefer
        system_prompt=build_student_prompt(profile), # one question in, one answer out
        output_type=str,                             # the student's answer
        deps_type=None,               
    )

convo = ConversationContext(transcript=[
    Message(who="teacher", content="Hello, what is projectile?"),
    Message(who="student", content="A projectile is an object given an initial velocity and acted upon by gravity, following a curved path.")
])

# agent = build_student_agent(students[0])
# result = await agent.run([convo.getConversationHistory(),"Cite an instance where this is useful"], 
#             deps=convo)
# print(result)
student_agents = []
for student in students:
    student_agents.append(build_student_agent(student))

for student_agent in student_agents:
    result = await student_agent.run([convo.getConversationHistory(),"Cite an instance where this is useful"])
    print(result)
    print("─" * 40)

16:09:51.998 student_agent run
16:09:51.998   chat gemini-2.5-flash
AgentRunResult(output="This is useful in sports, for example, calculating the trajectory of a basketball shot or a thrown football to improve accuracy. It's also critical in engineering, like designing the path for a water fountain or a rocket.")
────────────────────────────────────────
16:10:01.106 student_agent run
16:10:01.106   chat gemini-2.5-flash
AgentRunResult(output="Oh, well, like in basketball! When you shoot the ball, you're trying to get it to follow a specific arc to go into the hoop.")
────────────────────────────────────────
16:10:07.856 student_agent run
16:10:07.856   chat gemini-2.5-flash
AgentRunResult(output="It's useful in sports, like when a basketball player shoots. They need to understand the arc the ball will take to make the shot.")
────────────────────────────────────────
16:10:17.007 student_agent run
16:10:17.022   chat gemini-2.5-flash
AgentRunResult(output="Um, I'm not sure. I don't know

In [17]:
class AssessmentStrategy(str, Enum):
    SOCRATIC = "socratic"
    HIGHER_ORDER = "higher order"
    SCAFFOLDED = "scaffolded"

class PersonaTone(str, Enum):
    STRICT = "strict"
    FRIENDLY = "friendly"
    CASUAL_GENZ = "casual_genz"
    ENCOURAGING = "encouraging"
    FORMAL = "formal"
    STORYTELLER = "storyteller"
    HUMOROUS = "humorous"
    NEUTRAL = "neutral"

class FeedbackApproach(str, Enum):
    FORMATIVE = "formative"
    SUMMATIVE = "summative"
    REFLECTIVE = "reflective"
    PRAISE_HEAVY = "praise heavy"
    TOUGH_LOVE = "tough love"
    QUESTION_FEEDBACK = "question feedback"
    HUMOROUS = "humorous"

class AssessmentMode(str, Enum):
    REVIEW = "review"
    EXAM = "exam"

class TeacherPersona(BaseModel):
    name: str
    strategy: AssessmentStrategy
    tone: PersonaTone
    feedback: FeedbackApproach
    extra_traits: str  # freeform text, renamed from 'traits'
    mode: AssessmentMode

    def get_description(self) -> str:
        desc = (
            f"You are a high school teacher using a {self.strategy.value} questioning style in {self.mode.value} mode. "
            f"You speak with a {self.tone.value.replace('_', ' ')} tone and give {self.feedback.value} feedback. "
        )
        if self.extra_traits:
            desc += f"Traits include: {self.extra_traits}.\n"
        return desc


In [18]:
def build_teacher_prompt(profile: TeacherPersona) -> str:
    strategy_map = {
        AssessmentStrategy.SOCRATIC: "Ask probing guides toward deeper understanding. Do not provide answers.",
        AssessmentStrategy.HIGHER_ORDER: "Ask open-ended questions that require explanation, comparison, or justification.",
        AssessmentStrategy.SCAFFOLDED: "If the student struggles, rephrase or break the question into simpler parts and encourage them to try again."
    }

    tone_map = {
        PersonaTone.STRICT: "Use a firm, serious tone.",
        PersonaTone.FRIENDLY: "Use a warm, approachable tone.",
        PersonaTone.CASUAL_GENZ: "Use genz, modern language with a fun tone. Keep it short.",
        PersonaTone.ENCOURAGING: "Be positive and supportive.",
        PersonaTone.FORMAL: "Use academic, professional phrasing.",
        PersonaTone.STORYTELLER: "Use analogies or storytelling.",
        PersonaTone.HUMOROUS: "Use wit or light humor.",
        PersonaTone.NEUTRAL: "Use a calm, objective tone."
    }

    mode_map = {
        AssessmentMode.EXAM: "Do not give hints, rephrasing, or context clues. Your job is only to assess what the student already knows.",
        AssessmentMode.REVIEW: "If the student struggles, gently guide them by rephrasing or hinting. Your goal is to help the student learn while being assessed."
    }

    return f"""
You are a high school teacher. You are currently holding a recitation session with a student about a specific topic, where you ask them to answer specific questions based on the assessment data.
You are talking to the student. Always address the student directly using 'you' instead of saying 'the student'. DO NOT refer to the student in third person.
You are responsible for assessing a student's knowledge and understanding using a {profile.strategy.value} approach.
The assessment consists of questions, formatted according to the provided assessment topic wrapped within <AssessmentTopic> tag.

IMPORTANT INSTRUCTIONS:
1. The assessment is composed of questions. Ask a particular question only once, clearly to the student.
2. If the student gives the correct answer, acknowledge it positively.
3. If the student gives an incorrect or incomplete answer, provide a helpful hint or guidance and ask them to try again. 
4. The hint should start with "Think of", "Try to imagine", or "Remember that". The hint should give at most 40% ofthe answer to the succeeding question. The hint should not give away the keyword and the entire answer. The hint should be an imperative sentence, not an interrogative sentence.
5. You may give a maximum of one hint per question. Wait for the student to respond after you give the hint. If the student still does not answer correctly after one hint, state the correct answer.
6. If the students says "I don't know" even after given a hint, state the correct answer.
7. Only move on to the next question if the student gives the correct answer or you provided the entire answer.
8. After a correct answer by the student, acknowledge it with a phrase like "Good Job", "Nice Work", "Correct". Strictly only one sentence is allowed in acknowledging the correct answer.
9. If the student already correctly answered a question, do not repeat it. Move on to the next one.
10. Do not say "the assessment is done". Only say "that's all for that question" or "that's it for that item".




{strategy_map[profile.strategy]}

Speak in a {profile.tone.value.replace('_', ' ')} tone.
{tone_map[profile.tone]}

{mode_map[profile.mode]}
"""


### Framework Documentation

You are a professor. You are currently holding a recitation session with a student about a specific topic, where you ask them to answer specific questions based on the assessment data. You are  responsible for assessing a student’s knowledge through a structured assessment. The assessment consists of main questions and follow-up questions, formatted according to the provided JSON Object. Your role is to guide the student through the assessment, encouraging them to think critically and helping them when needed. 

Instructions for Conducting the Assessment:  

1. Ask Questions Clearly:  
- Present each main question one at a time from the given questions.  
- Format the question in a conversational tone suitable for verbal reading.  
- Ensure mathematical expressions are written in full English (e.g., “Alpha times x squared equals four” instead of “ax^2 = 4”). 

2. Validate Answers Based on Criteria:  
- When the student provides an answer, check if it meets the criteria.  
- The provided answer in the assessment data is a verified example, but not the only correct answer.  
- If the student gives an answer that aligns with the criteria, acknowledge it positively and move to the next question.  
- If the answer is incorrect or incomplete, select an appropriate follow-up question to guide them toward the right answer.  

3. Follow-Up Questions:  
- Clearly distinguish between a main question and a follow-up question.  
- Use follow-ups only if the student’s answer does not fully meet the criteria.  
- Encourage critical thinking instead of simply giving away the answer.  
- If the student is struggling, provide hints or rephrase the question for clarity.  

4. Handling Uncertainty:  
- If the student says they do not know the answer, gently nudge them with a hint.  
- Example: If the question is “What is the capital of France?” and they are unsure, say:  
  - “Think of a famous city known for the Eiffel Tower.”  

5. Concluding the Assessment:  
- Once all main questions have been answered, give a warm and encouraging goodbye message.  
- The goodbye message must contain the phrase “Good Bye” explicitly.  
- Example:  
  - “Great job today! You’ve done well in this assessment. Keep practicing and learning. Good Bye!”  

Response Formatting Example:  

Main Question:  
“What is the square root of sixteen?”  

Student’s Incorrect Response:  
"Maybe five?"  

Follow-Up Question:  
“That’s close, but not quite! Try thinking of a number that, when multiplied by itself, gives sixteen.”  

Student’s Correct Response:  
"Oh, it’s four!"  

AI Response:  
"That’s absolutely right! The square root of sixteen is four. Let’s move on to the next question."  

At all times, be patient, encouraging, and adaptable to the student's responses. Now, begin the assessment with the following assessment data: {{assessment_data}}.

In [19]:
def build_teacher_agent(profile: TeacherPersona) -> Agent:
    return Agent(
        model="google-gla:gemini-2.5-flash",
        system_prompt=build_teacher_prompt(profile),
        output_type=str,
        deps_type=None
    )

In [20]:
print(assessments['assessments'][0].questions[0])

text='Define Projectile and Projectile Motion.' answer='A projectile is an object given an initial velocity and acted upon solely by gravity in a curved path. Projectile motion is the form of motion exhibited by such an object.' criteria=['Answer must define both projectile and projectile motion, highlighting the role of initial velocity and gravity.'] isSelected=False followUps=[FollowUpQuestion(text="What is the primary force acting on a projectile once it's launched?", answer='Gravity is the primary force.', criteria=['Answer must identify gravity as the primary force.'], isSelected=False)]


In [21]:
question

Question(text='Explain how vertical velocity changes during the flight of an angle-launched projectile.', answer='As an angle-launched projectile ascends, its vertical velocity decreases until it reaches zero at the maximum height. As it descends, its vertical velocity increases due to gravity.', criteria=['Answer must describe the change in vertical velocity during ascent, at peak height, and during descent for an angle-launched projectile.'], isSelected=False, followUps=[FollowUpQuestion(text='What is the vertical velocity of a projectile at the peak of its trajectory?', answer='The vertical velocity of a projectile at the peak of its trajectory is zero.', criteria=['Answer must state that vertical velocity is zero at the peak.'], isSelected=False)])

## Step 5: Testing Different Combinations

### Finding Good Teacher-Student Matches

This section tests different combinations of teachers and students to see which ones work well together and create good learning conversations.

#### How We Test:

1. **Try Each Teacher**: Test every teacher style with sample questions
2. **Check Quality**: See if the responses sound natural and helpful
3. **Measure Learning**: Check if students actually learn from the interactions
4. **Test Flow**: Make sure conversations feel natural and engaging

#### What Makes a Good Combination:

- **Sounds Real**: The teacher and student responses match their personalities
- **Actually Helpful**: Students learn something from the conversation
- **Right Tone**: The teacher's style fits what the student needs
- **Smooth Flow**: The conversation moves naturally from question to answer

In [22]:
def format_question(question) -> str:
    lines = []
    lines.append("<AssessmentTopic>")
    lines.append(f"Question: {question.text}")
    lines.append(f"Example Answer: {question.answer}")
    lines.append(f"Criteria: {question.criteria}")
    for follow_up in question.followUps:
        lines.append(f"    >> Follow-up Question: {follow_up.text}")
        lines.append(f"    >> Example Answer: {follow_up.answer}")
        lines.append(f"    >> Criteria: {follow_up.criteria}")
    lines.append("</AssessmentTopic>")
    return "\n".join(lines)


test_assessor = build_teacher_agent(TeacherPersona(
        name="Casual Buddy",
        strategy=AssessmentStrategy.SCAFFOLDED,
        tone=PersonaTone.FORMAL,
        feedback=FeedbackApproach.REFLECTIVE,
        extra_traits="Relaxed, relatable, uses humor and modern language",
        mode=AssessmentMode.REVIEW
    ))

convo = ConversationContext(transcript=[
    Message(who="teacher", content="Hello, what is projectile?"),
    Message(who="student", content="A projectile is an object given an initial velocity and acted upon by gravity, following a curved path.")
])

question = assessments['assessments'][0].questions[0]
assessment_context = format_question(question)
context = [assessment_context,question.text, convo.getConversationHistory(), "Assess student answer."]
print(context)

result = await test_assessor.run(context)
print(result)

["<AssessmentTopic>\nQuestion: Define Projectile and Projectile Motion.\nExample Answer: A projectile is an object given an initial velocity and acted upon solely by gravity in a curved path. Projectile motion is the form of motion exhibited by such an object.\nCriteria: ['Answer must define both projectile and projectile motion, highlighting the role of initial velocity and gravity.']\n    >> Follow-up Question: What is the primary force acting on a projectile once it's launched?\n    >> Example Answer: Gravity is the primary force.\n    >> Criteria: ['Answer must identify gravity as the primary force.']\n</AssessmentTopic>", 'Define Projectile and Projectile Motion.', '<ConversationHistory> teacher: Hello, what is projectile?\nstudent: A projectile is an object given an initial velocity and acted upon by gravity, following a curved path. </ConversationHistory>', 'Assess student answer.']
16:11:57.859 test_assessor run
16:11:57.859   chat gemini-2.5-flash
AgentRunResult(output="That's

In [23]:
teachers = [
    TeacherPersona(
        name="Socratic Prober",
        strategy=AssessmentStrategy.SOCRATIC,
        tone=PersonaTone.FORMAL,
        feedback=FeedbackApproach.QUESTION_FEEDBACK,
        extra_traits="Encourages deep reasoning, pushes critical thinking, rarely gives hints",
        mode=AssessmentMode.EXAM,
        is_core=True
    ),
    TeacherPersona(
        name="Supportive Scaffolder",
        strategy=AssessmentStrategy.SCAFFOLDED,
        tone=PersonaTone.ENCOURAGING,
        feedback=FeedbackApproach.FORMATIVE,
        extra_traits="Guides gently, rephrases questions, reduces anxiety",
        mode=AssessmentMode.REVIEW,
        is_core=True
    ),
    TeacherPersona(
        name="Higher-Order Facilitator",
        strategy=AssessmentStrategy.HIGHER_ORDER,
        tone=PersonaTone.NEUTRAL,
        feedback=FeedbackApproach.REFLECTIVE,
        extra_traits="Asks open-ended, comparative, and analytical questions",
        mode=AssessmentMode.EXAM,
        is_core=True
    ),
    TeacherPersona(
        name="Structured Checker",
        strategy=AssessmentStrategy.SCAFFOLDED,
        tone=PersonaTone.STRICT,
        feedback=FeedbackApproach.SUMMATIVE,
        extra_traits="Efficient and consistent, focused on correctness and clarity",
        mode=AssessmentMode.EXAM
    ),
    TeacherPersona(
        name="Encouraging Coach",
        strategy=AssessmentStrategy.SCAFFOLDED,
        tone=PersonaTone.FRIENDLY,
        feedback=FeedbackApproach.PRAISE_HEAVY,
        extra_traits="Boosts confidence, praises effort, promotes participation",
        mode=AssessmentMode.REVIEW
    ),
    TeacherPersona(
        name="Casual Buddy",
        strategy=AssessmentStrategy.SCAFFOLDED,
        tone=PersonaTone.CASUAL_GENZ,
        feedback=FeedbackApproach.HUMOROUS,
        extra_traits="Relaxed, relatable, uses modern language",
        mode=AssessmentMode.REVIEW
    ),
    TeacherPersona(
        name="Analytical Guide",
        strategy=AssessmentStrategy.HIGHER_ORDER,
        tone=PersonaTone.FORMAL,
        feedback=FeedbackApproach.REFLECTIVE,
        extra_traits="Examines reasoning rigorously, links ideas, challenges assumptions",
        mode=AssessmentMode.EXAM
    ),
    TeacherPersona(
        name="Story-Based Mentor",
        strategy=AssessmentStrategy.SCAFFOLDED,
        tone=PersonaTone.STORYTELLER,
        feedback=FeedbackApproach.REFLECTIVE,
        extra_traits="Uses analogies, relatable scenarios, and narrative hooks to assess",
        mode=AssessmentMode.REVIEW
    )
]

In [24]:
convo = ConversationContext(transcript=[
    Message(who="teacher", content="Hello, what is projectile?"),
    Message(who="student", content="I am not sure."),
    Message(who="teacher", content="How about projectile motion?"),
    Message(who="student", content="Projectile motion describes the motion of an object that is launched into the air and only influenced by gravity, following a curved path"),
    Message(who="teacher", content="Now cite real world application of this concept?"),
    Message(who="student", content="It's useful in sports, like when a basketball player takes a shot. Understanding projectile motion helps them determine the right arc to get the ball into the hoop.")
])

teacher_agents = []
for teacher in teachers:
    teacher_agents.append(build_teacher_agent(teacher))

question = assessments['assessments'][0].questions[0]
assessment_context = format_question(question)
context = [assessment_context,question.text, convo.getConversationHistory(), "Assess student answer."]

for teacher_agent in teacher_agents:
    result = await teacher_agent.run(context)
    print(result)
    print("─" * 40)

16:12:12.193 teacher_agent run
16:12:12.193   chat gemini-2.5-flash
AgentRunResult(output="Let's start with the definition of a projectile. Think of what kind of object is involved in projectile motion.")
────────────────────────────────────────
16:12:15.594 teacher_agent run
16:12:15.594   chat gemini-2.5-flash
AgentRunResult(output="You've given a great explanation of projectile motion! Now, let's go back to the first part of the question. What is a **projectile** itself?")
────────────────────────────────────────
16:12:17.927 teacher_agent run
16:12:17.927   chat gemini-2.5-flash
AgentRunResult(output='You have provided a good description of projectile motion. However, your answer needs to include the definition of a projectile itself, and also mention how the motion begins. Remember that a projectile is a specific type of object, and its motion begins with a starting push. Please try again.')
────────────────────────────────────────
16:12:24.772 teacher_agent run
16:12:24.772   cha

## Step 6: Putting It All Together

### The Complete Process

This section shows how all the previous steps work together to create assessments from educational documents. It's like an assembly line that turns textbooks into conversation-ready questions.

#### The Complete Process:

1. **Load Document**: Take a PDF textbook or lesson material
2. **Find Topics**: Break it down into main subjects
3. **Create Questions**: Generate questions for each topic
4. **Save Everything**: Store all the questions in a file for later use

#### Creating New Assessments:

The code below shows how to make fresh questions from any educational document. You can uncomment these sections to create new question sets from different materials.

In [24]:
# import json

# FILE_URL = "./data/grade_9/science/SCI9-Q4-MOD1-Projectile Motion.pdf"
#split_result = await split_topics(AssessmentState(file_url=FILE_URL))
#assessment_result = await generate_multiple_assessments(AssessmentState(topics=split_result['topics'], title=split_result['title']))

#def custom_encoder(obj):
#    if hasattr(obj, "to_dict"):
#        return obj.to_dict()
#    return obj.__dict__  # fallback: serialize __dict__

#with open(f"{FILE_URL}.json", "w") as f:
#   json.dump(assessment_result, f, default=custom_encoder, indent=4)


## Step 7: Testing One-on-One Conversations

### Simple Teacher-Student Test

This section shows a conversation between just one teacher and one student. It's like a practice run to make sure everything works before we create lots of conversations.

#### What We're Testing:

- **Teacher**: Usually the "Questioning Teacher" (asks lots of follow-up questions)
- **Student**: Usually the "Focused Student" (good at solving problems step-by-step)
- **Questions**: All the questions we created from the lesson material

#### Why This Helps:

1. **Check Flow**: See if the conversation feels natural
2. **Test Responses**: Make sure both teacher and student sound realistic
3. **Find Problems**: Catch any issues before making lots of conversations
4. **Improve Performance**: Fine-tune how the AI characters behave

In [25]:
from __future__ import annotations
import os
import csv
import json
import asyncio
from dataclasses import dataclass, field
from typing import Union, Optional
from pydantic_graph import BaseNode, End, Graph, GraphRunContext
import random


ASSESSMENT_JSON_PATH = "./data/grade_9/science/SCI9-Q4-MOD1-Projectile Motion.pdf.json"
with open(ASSESSMENT_JSON_PATH, "r", encoding="utf-8") as f:
    assessment_result = json.load(f)

print(f"Loaded assessment data from {ASSESSMENT_JSON_PATH}")

Loaded assessment data from ./data/grade_9/science/SCI9-Q4-MOD1-Projectile Motion.pdf.json


In [ ]:
@dataclass
class Question:
    text: str
    followUps: list = field(default_factory=list)

    @staticmethod
    def from_dict(d: dict) -> 'Question':
        follow_ups = [Question.from_dict(fu) for fu in d.get("followUps", [])]
        return Question(text=d["text"], followUps=follow_ups)

@dataclass
class Message:
    who: str
    content: str

# Holds the entire state of the conversation
@dataclass
class ConversationContext:
    question: Optional[Question] = None
    transcript: list[Message] = field(default_factory=list)
    main_question_attempts: int = 0
    question_index: int = 0
    questions: list = field(default_factory=list)
    conversation_done: bool = False
    teacher_turn_stage: int = 0  
    exchanges_for_current_question: int = 0

    # Returns formatted history of the entire conversation so far
    def getConversationHistory(self) -> str:
        history = "\n".join(f"{msg.who}: {msg.content}" for msg in self.transcript)
        return f"<ConversationHistory> {history} </ConversationHistory>"

    # Adds a message to the conversation transcript
    def addMessage(self, message: Message):
        self.transcript.append(message)

# Retry wrapper for agent calls

async def safe_run(agent, context, retries=3, delay=3):
    for attempt in range(retries):
        try:
            return await agent.run(context)
        except Exception as e:
            print(f"ERROR: {e} - Retrying ({attempt+1}/{retries})...")
            if attempt < retries - 1:
                await asyncio.sleep(delay)
    raise RuntimeError(f"Failed after {retries} attempts.")


# Teacher Node

@dataclass
class Teacher(BaseNode[ConversationContext]):
    closing_phrases = [
        "Alright, moving to the next one.",
        "Let's continue with the next question.",
        "Good, let's move on."
    ]

    async def run(self, ctx: GraphRunContext) -> Union["Student", End]:
        print('------------------- TEACHER --------------------')
        if ctx.state.conversation_done:
            return End(1)
        return await self.handle_question(ctx)

    async def handle_question(self, ctx: GraphRunContext):
        if ctx.state.teacher_turn_stage == 0:
            # Ask the main question
            question_text = ctx.state.question.text
            print('Teacher:', question_text)
            ctx.state.addMessage(Message(who="teacher", content=question_text))
            ctx.state.teacher_turn_stage = 1
            return Student()
        elif ctx.state.teacher_turn_stage in (1, 2):
            # Evaluate or give hint
            return await self.evaluate_answer(ctx)

    async def evaluate_answer(self, ctx: GraphRunContext):
        context = [
            ctx.state.question.text,
            ctx.state.getConversationHistory(),
            "Evaluate the student's answer briefly. Do NOT ask a new question."
        ]
        reply = await safe_run(teacher_a, context)
        teacher_reply = reply.output.strip() if reply and reply.output else "Let's move on."

        # Ensure teacher doesn't ask another question
        if "?" in teacher_reply:
            teacher_reply = teacher_reply.split("?")[0].strip() + "."

        print('Teacher:', teacher_reply)
        ctx.state.addMessage(Message(who="teacher", content=teacher_reply))

        # Check for correctness
        positive_keywords = ['correct', 'right', 'good job', 'excellent', 'perfect', 'nice work']
        has_positive_keyword = any(keyword in teacher_reply.lower() for keyword in positive_keywords)

        if has_positive_keyword:
            # If not the last question, add a transition phrase
            if ctx.state.question_index < len(ctx.state.questions) - 1:
                closing = random.choice(self.closing_phrases)
                ctx.state.addMessage(Message(who="teacher", content=closing))
            ctx.state.teacher_turn_stage = 0
            return self.move_to_next_question(ctx)

        # Handle hint or move on logic
        if ctx.state.teacher_turn_stage == 1:
            print("DEBUG: Giving a hint.")
            ctx.state.teacher_turn_stage = 2
            return Student()
        else:
            print("DEBUG: Student failed after hint. Moving to next question.")
            ctx.state.teacher_turn_stage = 0
            return self.move_to_next_question(ctx)

    def move_to_next_question(self, ctx: GraphRunContext):
        ctx.state.question_index += 1
        if ctx.state.question_index >= len(ctx.state.questions):
            # End of all questions
            print("DEBUG: All questions completed. Ending conversation.")
            ctx.state.addMessage(Message(who="teacher", content="Goodbye! Have a great day."))
            ctx.state.conversation_done = True
            return End(1)
        # Load the next question
        ctx.state.question = ctx.state.questions[ctx.state.question_index]
        ctx.state.teacher_turn_stage = 0
        return Teacher()

# Student Node

@dataclass
class Student(BaseNode[ConversationContext]):
    async def run(self, ctx: GraphRunContext) -> "Teacher":
        print('------------------- STUDENT --------------------')
        context = [ctx.state.getConversationHistory()]
        try:
            reply = await safe_run(student_a, context)
            student_reply = reply.output.strip() if reply and reply.output else "I don't know."
        except Exception as e:
            print(f"ERROR: Student failed to respond - {e}")
            student_reply = "I don't know."

        print('    Student:', student_reply)
        ctx.state.addMessage(Message(who="student", content=student_reply))
        return Teacher()

# Build Graph of Conversation Flow

conversation_graph = Graph(
    nodes=[Teacher, Student],
    state_type=ConversationContext,
)

# Flatten nested question structure

def flatten_questions(assessments):
    all_questions = []
    for assessment in assessments:
        for q in assessment["questions"]:
            q_obj = Question.from_dict(q)
            all_questions.append(q_obj)
            all_questions.extend(q_obj.followUps)
    return all_questions

# Run for only Teacher 1 and Student 1

async def run_single_teacher_student():
    global teacher_a, student_a

    teacher = teachers[0]
    student = students[0]

    teacher_a = build_teacher_agent(teacher)
    student_a = build_student_agent(student)

    all_questions = flatten_questions(assessment_result['assessments'])

    # Create initial conversation context
    conversation = ConversationContext(
        transcript=[],
        question=all_questions[0],
        questions=all_questions,
        question_index=0
    )

    # Run the conversation graph
    print(f"\n=== STARTING CONVERSATION: {teacher.name} + {student.profile} ===")
    await conversation_graph.run(Teacher(), state=conversation)
    print(f"=== CONVERSATION COMPLETE ===")


# Run the conversation

await run_single_teacher_student()


=== STARTING CONVERSATION: Socratic Prober + a grade 9 high school student who is a focused problem-solver with high logic and structure ===
15:53:35.262 run graph conversation_graph
15:53:35.264   run node Teacher
------------------- TEACHER --------------------
Teacher: Define projectile motion and a projectile.
15:53:35.264   run node Student
------------------- STUDENT --------------------
15:53:35.264     agent run
15:53:35.264       chat gemini-2.5-flash
    Student: Okay, so projectile motion is the movement of an object where the only force acting on it is gravity. And a projectile is that object itself.
15:53:37.605   run node Teacher
------------------- TEACHER --------------------
15:53:37.605     agent run
15:53:37.605       chat gemini-2.5-flash
Teacher: Correct.
15:53:39.120   run node Teacher
------------------- TEACHER --------------------
Teacher: What is the primary force acting on a projectile after it's launched?
15:53:39.120   run node Student
------------------- 

## Step 8: Creating All Possible Conversations

### Making Every Teacher Talk to Every Student

This section creates conversations between every teacher type and every student type. It's like having a big classroom where every teacher gets to work with every student.

#### What We're Creating:

- **8 Different Teachers**: Each with their own style (strict, friendly, etc.)
- **9 Different Students**: Each with their own personality and ability level
- **72 Total Conversations**: Every possible teacher-student combination
- **Same Questions**: All conversations use the questions from our lesson material

#### How It Works:

1. **Goes Through Everyone**: Creates conversations systematically for all combinations
2. **Saves Each One**: Every conversation gets saved as its own file
3. **Tracks Progress**: Shows you how many are done and how many are left
4. **Handles Problems**: If one conversation fails, it keeps going with the others

#### How We Organize the Results:

- **Clear File Names**: Each file is named like `conversation_teacher_1_student_3.csv`
- **Includes Info**: Each file shows which teacher and student personalities were used
- **Easy to Read**: Saved in a simple format that's easy to analyze
- **Ready for Audio**: Formatted so it can easily be turned into speech

In [ ]:
ASSESSMENT_JSON_PATH = "./data/grade_9/science/SCI9-Q4-MOD1-Projectile Motion.pdf.json"

with open(ASSESSMENT_JSON_PATH, "r", encoding="utf-8") as f:
    assessment_result = json.load(f)

print(f"Loaded assessment data from {ASSESSMENT_JSON_PATH}")


# Define Question wrapper

@dataclass
class Question:
    text: str
    followUps: list = field(default_factory=list)

    @staticmethod
    def from_dict(d: dict) -> 'Question':
        follow_ups = [Question.from_dict(fu) for fu in d.get("followUps", [])]
        return Question(text=d["text"], followUps=follow_ups)


# Retry wrapper for model calls

async def safe_run(agent, context, retries=3, delay=3):
    for attempt in range(retries):
        try:
            return await agent.run(context)
        except Exception as e:
            print(f"ERROR: {e} - Retrying ({attempt+1}/{retries})...")
            if attempt < retries - 1:
                await asyncio.sleep(delay)
    raise RuntimeError(f"Failed after {retries} attempts.")


# Data Models

@dataclass
class Message:
    who: str
    content: str

@dataclass
class ConversationContext:
    question: Optional[Question] = None
    transcript: list[Message] = field(default_factory=list)
    main_question_attempts: int = 0
    question_index: int = 0
    questions: list = field(default_factory=list)
    conversation_done: bool = False
    teacher_turn_stage: int = 0
    exchanges_for_current_question: int = 0

    def getConversationHistory(self) -> str:
        history = "\n".join(f"{msg.who}: {msg.content}" for msg in self.transcript)
        return f"<ConversationHistory> {history} </ConversationHistory>"

    def addMessage(self, message: Message):
        self.transcript.append(message)


# Teacher Node

@dataclass
class Teacher(BaseNode[ConversationContext]):
    closing_phrases = [
        "Alright, moving to the next one.",
        "Let's continue with the next question.",
        "Good, let's move on.",
        "Next up, here's another one.",
        "Okay, here's the next question.",
        "Let's try the following question.",
        "We're moving ahead now.",
        "Time for the next one.",
        "Let's proceed to the next item.",
        "Here comes the next question."
        
    ]

    async def run(self, ctx: GraphRunContext) -> Union["Student", End]:
        print('------------------- TEACHER --------------------')
        if ctx.state.conversation_done:
            return End(1)
        return await self.handle_question(ctx)

    async def handle_question(self, ctx: GraphRunContext):
        if ctx.state.teacher_turn_stage == 0:
            question_text = ctx.state.question.text
            print('Teacher:', question_text)
            ctx.state.addMessage(Message(who="teacher", content=question_text))
            ctx.state.teacher_turn_stage = 1
            return Student()
        elif ctx.state.teacher_turn_stage in (1, 2):
            return await self.evaluate_answer(ctx)

    async def evaluate_answer(self, ctx: GraphRunContext):
        # Ensure teacher only evaluates, no new questions
        context = [
            ctx.state.question.text,
            ctx.state.getConversationHistory(),
            "Evaluate the student's answer briefly. Do NOT ask a new question."
        ]
        reply = await safe_run(teacher_a, context)
        teacher_reply = reply.output.strip() if reply and reply.output else "Let's move on."

        # Prevent accidental new question
        if "?" in teacher_reply:
            teacher_reply = teacher_reply.split("?")[0].strip() + "."

        print('Teacher:', teacher_reply)
        ctx.state.addMessage(Message(who="teacher", content=teacher_reply))

        positive_keywords = ['correct', 'right', 'good job', 'excellent', 'perfect', 'nice work']
        has_positive_keyword = any(keyword in teacher_reply.lower() for keyword in positive_keywords)

        if has_positive_keyword:
            # Add closing phrase except for the last question
            if ctx.state.question_index < len(ctx.state.questions) - 1:
                closing = random.choice(self.closing_phrases)
                ctx.state.addMessage(Message(who="teacher", content=closing))
            ctx.state.teacher_turn_stage = 0
            return self.move_to_next_question(ctx)

        if ctx.state.teacher_turn_stage == 1:
            print("DEBUG: Giving a hint.")
            ctx.state.teacher_turn_stage = 2
            return Student()
        else:
            print("DEBUG: Student failed after hint. Moving to next question.")
            ctx.state.teacher_turn_stage = 0
            return self.move_to_next_question(ctx)

    def move_to_next_question(self, ctx: GraphRunContext):
        ctx.state.question_index += 1
        if ctx.state.question_index >= len(ctx.state.questions):
            print("DEBUG: All questions completed. Ending conversation.")
            ctx.state.addMessage(Message(who="teacher", content="Goodbye! Have a great day."))
            ctx.state.conversation_done = True
            return End(1)
        ctx.state.question = ctx.state.questions[ctx.state.question_index]
        ctx.state.teacher_turn_stage = 0
        return Teacher()


# Student Node

@dataclass
class Student(BaseNode[ConversationContext]):
    async def run(self, ctx: GraphRunContext) -> "Teacher":
        print('------------------- STUDENT --------------------')
        context = [ctx.state.getConversationHistory()]
        try:
            reply = await safe_run(student_a, context)
            student_reply = reply.output.strip() if reply and reply.output else "I don't know."
        except Exception as e:
            print(f"ERROR: Student failed to respond - {e}")
            student_reply = "I don't know."

        print('    Student:', student_reply)
        ctx.state.addMessage(Message(who="student", content=student_reply))
        return Teacher()


# Graph

conversation_graph = Graph(
    nodes=[Teacher, Student],
    state_type=ConversationContext,
)


# Flatten Questions

def flatten_questions(assessments):
    all_questions = []
    for assessment in assessments:
        for q in assessment["questions"]:
            q_obj = Question.from_dict(q)
            all_questions.append(q_obj)
            all_questions.extend(q_obj.followUps)
    return all_questions


# Runner Function

async def run_all_teachers_all_students(start_teacher=4, start_student=9):
    os.makedirs("conversations", exist_ok=True)

    for t_idx, teacher in enumerate(teachers[start_teacher-1:], start=start_teacher):
        global teacher_a
        teacher_a = build_teacher_agent(teacher)

        student_start = start_student if t_idx == start_teacher else 1

        for s_idx, student in enumerate(students[student_start-1:], start=student_start):
            global student_a
            student_a = build_student_agent(student)

            all_questions = flatten_questions(assessment_result['assessments'])

            conversation = ConversationContext(
                transcript=[],
                question=all_questions[0],
                questions=all_questions,
                question_index=0
            )

            print(f"\n=== STARTING CONVERSATION: {teacher.name} + {student.profile} ===")
            await conversation_graph.run(Teacher(), state=conversation)
            print(f"=== CONVERSATION COMPLETE ===")

            filename = f"conversations/conversation_teacher_{t_idx}_student_{s_idx}.csv"
            with open(filename, 'w', newline='', encoding='utf-8') as csvfile:
                writer = csv.writer(csvfile)
                writer.writerow([f"Conversation with Teacher {t_idx} and Student {s_idx}"])
                writer.writerow([f"teacher profile: {teacher.name}"])
                writer.writerow([f"student profile: {student.profile}"])
                writer.writerow([])

                for msg in conversation.transcript:
                    writer.writerow([msg.who, msg.content])

            print(f"Conversation saved to {filename}")


# Start Execution

await run_all_teachers_all_students(start_teacher=1, start_student=1)

## Step 9: Making Questions Match Teacher Personalities

### Personalizing How Teachers Ask Questions

This section improves the system by making each teacher ask questions in their own unique way. Instead of all teachers asking the same question the same way, each one phrases it according to their personality.

#### What We're Improving:

1. **Personal Question Style**: Each teacher asks questions in their own way
2. **Natural Communication**: Questions sound like they really come from that type of teacher
3. **Consistent Teaching**: Teachers stick to their style throughout the conversation
4. **Same Voice**: Each teacher maintains their personality in every interaction

#### How We Do This:

- **Rewrite Questions**: Have each teacher rephrase the same question in their style
- **Keep It Accurate**: Don't change the meaning, just the way it's asked
- **Do It for Everyone**: Create personalized versions for all teacher types
- **Use Later**: Integrate these personalized questions into future conversations

#### Why This Makes Things Better:

- **More Realistic**: Conversations sound more like real teacher-student interactions
- **Better Engagement**: Students respond better when questions match the teacher's style
- **Better Audio**: When converted to speech, it sounds more natural
- **More Variety**: Creates a richer collection of different ways to ask the same thing

In [ ]:
import json
import csv
from dataclasses import dataclass, field

# Question Class
@dataclass
class Question:
    text: str
    followUps: list = field(default_factory=list)

    @staticmethod
    def from_dict(d: dict) -> 'Question':
        follow_ups = [Question.from_dict(fu) for fu in d.get("followUps", [])]
        return Question(text=d["text"], followUps=follow_ups)

# Flatten
def flatten_questions(assessments):
    all_questions = []
    for assessment in assessments:
        for q in assessment["questions"]:
            q_obj = Question.from_dict(q)
            all_questions.append(q_obj)
            all_questions.extend(q_obj.followUps)
    return all_questions


ASSESSMENT_JSON_PATH = "./data/grade_9/science/SCI9-Q4-MOD1-Projectile Motion.pdf.json"
with open(ASSESSMENT_JSON_PATH, "r", encoding="utf-8") as f:
    assessment_result = json.load(f)

# Reword and Save Function
async def reword_all_questions_by_all_teachers():
    all_questions = flatten_questions(assessment_result["assessments"])
    csv_rows = []

    for i, question in enumerate(all_questions):
        print(f"\n=========== Question {i+1} ===========")
        print(f"Original: {question.text}")

        for j, teacher in enumerate(teachers):
            teacher_agent = build_teacher_agent(teacher)

            assessment_input = f"""
<AssessmentTopic>
Question: {question.text}
</AssessmentTopic>

Rewrite the question below in a way that fits your teacher personality, 
but keep the rewording minimal so that the meaning doesn't change.

Speak as if you're talking directly to a student. Do not include greetings, introductions, or formalities.
Do not say “Let’s begin,” “Good morning,” etc. Just ask the question.
Avoid removing important scientific terms or detail.

Question: {question.text}
"""

            try:
                result = await teacher_agent.run(assessment_input)
                reworded = result.output.strip()
                print(f"\n--- Teacher {j+1}: {teacher.name} ---")
                print(reworded)
                csv_rows.append([f"teacher_{j+1}", question.text, reworded])
            except Exception as e:
                print(f"Error with Teacher {teacher.name}: {e}")
                csv_rows.append([f"teacher_{j+1}", question.text, f"[Error: {e}]"])

    # Save to CSV
    output_path = "questions-with-teacher-personalities.csv"
    with open(output_path, "w", newline="", encoding="utf-8") as f:
        writer = csv.writer(f)
        writer.writerow(["teacher_n", "original_question", "modified_question"])
        writer.writerows(csv_rows)

    print(f"\nSaved to {output_path}")

# Run
await reword_all_questions_by_all_teachers()

## Step 10: Testing the Complete System

### Final Test with Different Student Types

This final section tests the complete system using personalized questions and shows how one teacher adapts to different types of students. It's like watching a teacher work with various students in their classroom.

#### What We're Testing:

- **Question Source**: Uses a simplified set of 3 questions with personalized teacher styles
- **Teacher**: The "Questioning Teacher" (asks follow-up questions to help students think)
- **Students**: Three different types representing various ability levels:
  - **Student 2**: Smart and focused, good at solving problems step-by-step
  - **Student 3**: Friendly and collaborative, learns at a moderate pace
  - **Student 6**: Less motivated and struggles more with the material

#### What We Want to See:

1. **Different Responses**: Each student type should respond differently to the same questions
2. **Teacher Adaptation**: The teacher should adjust their approach for each student
3. **Natural Flow**: Conversations should feel realistic and educational
4. **Audio Ready**: Everything should be ready to convert to spoken conversations

#### What Should Happen:

- **Varied Conversations**: Each student will have a different conversation pattern
- **Consistent Teacher**: The teacher maintains their questioning style with all students
- **Good Teaching**: The teacher provides appropriate help for each student's level
- **High Quality**: The conversations should be good enough to use in real educational apps

In [27]:
import json
import os
import asyncio
import random
from dataclasses import dataclass, field
from typing import Optional, Union, List, Dict, Any

# Question Class
@dataclass
class Question:
    text: str
    answer: str
    criteria: List[str]
    followUps: list = field(default_factory=list)

    @staticmethod
    def from_dict(d: dict) -> 'Question':
        follow_ups = [Question.from_dict(fu) for fu in d.get("followUps", [])]
        return Question(
            text=d["text"], 
            answer=d["answer"],
            criteria=d["criteria"],
            followUps=follow_ups
        )

# Tree Node for conversation structure
@dataclass
class ConversationNode:
    speaker: str
    message: str
    tag: str
    audio_link: str
    responses: List['ConversationNode'] = field(default_factory=list)

    def to_dict(self) -> Dict[str, Any]:
        return {
            "speaker": self.speaker,
            "message": self.message,
            "tag": self.tag,
            "audio link": self.audio_link,
            "responses": [response.to_dict() for response in self.responses]
        }

# Flatten Questions
def flatten_questions(assessments):
    all_questions = []
    for assessment in assessments:
        for q in assessment["questions"]:
            q_obj = Question.from_dict(q)
            all_questions.append(q_obj)
    return all_questions

# Safe Run
async def safe_run(agent, context, retries=3, delay=3):
    for attempt in range(retries):
        try:
            return await agent.run(context)
        except Exception as e:
            print(f"ERROR: {e} - Retrying ({attempt+1}/{retries})...")
            if attempt < retries - 1:
                await asyncio.sleep(delay)
    raise RuntimeError(f"Failed after {retries} attempts.")

# Tag generator for unique conversation identifiers
class TagGenerator:
    def __init__(self):
        self.counters = {}
    
    def generate_tag(self, speaker: str, question_num: int, interaction_type: str = "response") -> str:
        key = f"{speaker}_{question_num}_{interaction_type}"
        if key not in self.counters:
            self.counters[key] = 0
        self.counters[key] += 1
        return f"{speaker}_question_{question_num}_{self.counters[key]:04d}"

# Audio link generator
def generate_audio_link(tag: str) -> str:
    return f"https://huggingface.co/datasets/itsybitsci/audio/resolve/main/{tag}.wav?download=true"

# Helper function to check if teacher indicates conversation should end
def should_end_conversation(teacher_response: str) -> bool:
    """Check if teacher's response indicates the conversation should end"""
    teacher_lower = teacher_response.lower()
    
    # Positive ending indicators
    positive_endings = [
        "that's correct", "that's right", "exactly", "perfect", "well done",
        "good job", "excellent", "you got it", "that's the answer",
        "correct!", "right!", "yes, that's it", "absolutely right",
        "great work", "outstanding", "brilliant", "spot on"
    ]
    
    # Check for positive endings
    for ending in positive_endings:
        if ending in teacher_lower:
            return True
    
    # Also check for final explanation patterns (when teacher gives the answer)
    final_patterns = [
        "the answer is", "the correct answer", "let me explain",
        "here's the solution", "the right answer"
    ]
    
    for pattern in final_patterns:
        if pattern in teacher_lower:
            return True
    
    return False

# Conversation Tree Builder
class ConversationTreeBuilder:
    def __init__(self, teacher_agent, student_agents, tag_generator):
        self.teacher_agent = teacher_agent
        self.student_agents = student_agents
        self.tag_generator = tag_generator
        self.current_question_num = 1
    
    async def build_question_tree(self, question: Question) -> Dict[str, Any]:
        """Build a complete conversation tree for a single question"""
        
        # Root question node
        question_node = {
            "speaker": "question",
            "message": question.text,
            "children": []
        }
        
        # Generate teacher's initial question presentation
        teacher_node = await self.generate_teacher_question(question)
        
        # Start the tree-like conversation from the teacher's initial question
        await self.generate_tree_responses(question, teacher_node, hint_level=1)
        
        question_node["children"].append(teacher_node.to_dict())
        return question_node
    
    async def generate_teacher_question(self, question: Question) -> ConversationNode:
        """Generate teacher's initial presentation of the question"""
        greeting = random.choice([
            "Good morning! Let's explore this together.",
            "Hello! Here's an interesting question for you.",
            "Hi there! I'd like you to think about something.",
            "Good day! Let's work through this together.",
            "Hello! I have a question for you."
        ])
        
        prompt = f"""
Rewrite the question below in a way that fits your teacher personality.
Start with this greeting phrase: {greeting}

Then add the reworded question. Keep the rewording minimal so the meaning doesn't change.
Avoid removing important scientific terms or detail.

Only use the greeting + reworded question. Do NOT add any other content.

Question: {question.text}
"""
        
        result = await safe_run(self.teacher_agent, [prompt])
        reworded_question = result.output.strip() if result and result.output else f"{greeting} {question.text}"
        
        tag = self.tag_generator.generate_tag("teacher_1", self.current_question_num, "question")
        
        return ConversationNode(
            speaker="teacher_1",
            message=reworded_question,
            tag=tag,
            audio_link=generate_audio_link(tag)
        )
    
    async def generate_tree_responses(self, question: Question, teacher_node: ConversationNode, hint_level: int):
        """Generate responses from ALL students to the teacher's message, creating a tree structure"""
        
        # Generate responses from ALL students to this teacher message
        for i, student_agent in enumerate(self.student_agents):
            student_num = [2, 3, 6][i]  # Students 2, 3, 6
            
            # Generate student response to the teacher's message
            context = [f"Teacher: {teacher_node.message}"]
            result = await safe_run(student_agent, context)
            student_response = result.output.strip() if result and result.output else "I'm not sure."
            
            tag = self.tag_generator.generate_tag(f"student_{student_num}", self.current_question_num, f"response_{hint_level}")
            student_node = ConversationNode(
                speaker=f"student_{student_num}",
                message=student_response,
                tag=tag,
                audio_link=generate_audio_link(tag)
            )
            
            # Add student response to teacher's responses
            teacher_node.responses.append(student_node)
            
            # Generate teacher's feedback to this specific student
            teacher_feedback = await self.generate_teacher_response(
                question, teacher_node.message, student_response, student_num, hint_level
            )
            
            # Add teacher feedback to student's responses
            student_node.responses.append(teacher_feedback)
            
            # If teacher doesn't indicate end of conversation, continue the tree
            if not should_end_conversation(teacher_feedback.message) and hint_level < 4:  # Limit depth to avoid infinite loops
                await self.generate_tree_responses(question, teacher_feedback, hint_level + 1)
    
    async def generate_teacher_response(self, question: Question, teacher_question: str, 
                                      student_response: str, student_num: int, hint_level: int) -> ConversationNode:
        """Generate teacher's response - could be approval, correction, hint, or final explanation"""
        
        # Determine the type of response needed based on hint level
        if hint_level == 1:
            instruction = """
Evaluate the student's response to the question. If it's correct or very close, give positive feedback like "That's correct!" or "Exactly right!" or "Well done!"

If the answer is incorrect or incomplete, give a gentle hint that guides the student toward the right thinking without giving away the answer. Ask a leading question or point them in the right direction.

Be encouraging and educational. Address the student directly.
"""
        elif hint_level <= 3:
            instruction = f"""
The student is still working on this question (attempt #{hint_level}). 

If their latest response shows they now understand and have the right answer, give positive feedback like "That's correct!" or "Perfect!" or "You got it!"

If they still need help, give a more specific hint. Provide key information or break down the problem into smaller parts. Make your hint more direct than before since this is attempt #{hint_level}.

Be encouraging and educational. Address the student directly.
"""
        else:
            instruction = """
The student has tried several times. At this point, either:
1. If their response shows understanding, give positive feedback like "That's correct!" or "Excellent work!"
2. If they're still struggling, provide a clear explanation with the correct answer to help them learn.

Be encouraging and educational. Address the student directly.
"""
        
        context = [
            f"Original question: {question.text}",
            f"Correct answer: {question.answer}",
            f"Evaluation criteria: {', '.join(question.criteria)}",
            f"Student {student_num} responded: {student_response}",
            instruction
        ]
        
        result = await safe_run(self.teacher_agent, context)
        response = result.output.strip() if result and result.output else "Let me help you think about this."
        
        # Determine tag type based on response content
        if should_end_conversation(response):
            tag_type = "approval" if any(word in response.lower() for word in ["correct", "right", "excellent", "perfect"]) else "final_explanation"
        else:
            tag_type = f"hint_{hint_level}"
        
        tag = self.tag_generator.generate_tag("teacher_1", self.current_question_num, tag_type)
        
        return ConversationNode(
            speaker="teacher_1",
            message=response,
            tag=tag,
            audio_link=generate_audio_link(tag)
        )

# Main Runner
async def generate_conversation_tree():
    """Generate complete conversation tree structure"""
    
    # Load questions
    with open("simplified/questions-3-items.json", "r", encoding="utf-8") as f:
        assessment_result = json.load(f)
    
    questions = flatten_questions(assessment_result["assessments"])
    
    # Setup agents
    global teacher_a
    teacher = teachers[0]  # Teacher 1
    teacher_a = build_teacher_agent(teacher)
    
    # Use students 2, 3, and 6 as specified in the instructions
    selected_student_indices = [1, 2, 5]  # 0-indexed: student 2, 3, 6
    student_agents = []
    for i in selected_student_indices:
        student = students[i]
        student_agent = build_student_agent(student)
        student_agents.append(student_agent)
    
    # Initialize tree builder
    tag_generator = TagGenerator()
    tree_builder = ConversationTreeBuilder(teacher_a, student_agents, tag_generator)
    
    # Build conversation tree
    conversation_tree = {
        "questions": []
    }
    
    for i, question in enumerate(questions):
        print(f"\n=== Generating conversation tree for question {i+1}: {question.text[:50]}... ===")
        tree_builder.current_question_num = i + 1
        
        question_tree = await tree_builder.build_question_tree(question)
        conversation_tree["questions"].append(question_tree)
        
        print(f"✓ Completed question {i+1}")
    
    # Save the tree
    output_dir = "simplified"
    os.makedirs(output_dir, exist_ok=True)
    
    output_path = os.path.join(output_dir, "conversation-tree-with-audio.json")
    with open(output_path, "w", encoding="utf-8") as f:
        json.dump(conversation_tree, f, indent=2, ensure_ascii=False)
    
    print(f"\n✓ Conversation tree saved to {output_path}")
    print(f"Generated {len(conversation_tree['questions'])} question trees")
    
    return conversation_tree

# Start generation
await generate_conversation_tree()


=== Generating conversation tree for question 1: Within the topic of projectile motion, how would y... ===
16:25:20.038 agent run
16:25:20.038   chat gemini-2.5-flash
16:25:24.332 agent run
16:25:24.342   chat gemini-2.5-flash
16:25:28.437 agent run
16:25:28.437   chat gemini-2.5-flash
16:25:42.418 agent run
16:25:42.418   chat gemini-2.5-flash
16:25:45.556 agent run
16:25:45.556   chat gemini-2.5-flash
16:25:48.996 agent run
16:25:48.998   chat gemini-2.5-flash
16:25:53.586 agent run
16:25:53.586   chat gemini-2.5-flash
16:25:55.283 agent run
16:25:55.283   chat gemini-2.5-flash
16:25:59.279 agent run
16:25:59.279   chat gemini-2.5-flash
16:26:01.115 agent run
16:26:01.115   chat gemini-2.5-flash
16:26:04.013 agent run
16:26:04.015   chat gemini-2.5-flash
16:26:10.336 agent run
16:26:10.339   chat gemini-2.5-flash
16:26:14.435 agent run
16:26:14.435   chat gemini-2.5-flash
16:26:15.666 agent run
16:26:15.666   chat gemini-2.5-flash
16:26:21.304 agent run
16:26:21.307   chat gemini-2.

{'questions': [{'speaker': 'question',
   'message': 'Within the topic of projectile motion, how would you describe a projectile?',
   'children': [{'speaker': 'teacher_1',
     'message': "Good morning! Let's explore this together. In the context of projectile motion, how would you define a projectile?",
     'tag': 'teacher_1_question_1_0001',
     'audio link': 'https://huggingface.co/datasets/itsybitsci/audio/resolve/main/teacher_1_question_1_0001.wav?download=true',
     'responses': [{'speaker': 'student_2',
       'message': "Okay, so a projectile is pretty much an object that's thrown or launched into the air, and then it just keeps moving, mostly because of gravity.",
       'tag': 'student_2_question_1_0001',
       'audio link': 'https://huggingface.co/datasets/itsybitsci/audio/resolve/main/student_2_question_1_0001.wav?download=true',
       'responses': [{'speaker': 'teacher_1',
         'message': 'You are correct that a projectile is an object that is thrown or launched 